In [1]:
from feature.make_dataset import make_data
from feature.after_dataset import accumulate
import pandas as pd
from glob import glob
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import glob
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')


c:\Python\Python38\lib\site-packages\xgboost\compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
train_input_list = './data/train_input/*.csv'
train_target_list = './data/train_target/*.csv'

test_input_list = './data/test_input/*.csv'
test_target_list = './data/test_target/*.csv'

In [3]:
train, test, p_train, p_test = make_data(train_input_list, train_target_list, test_input_list, test_target_list)

In [4]:
train

,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,...,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량,predicted_weight_g,Case,6time,diff_temp,dark,water
0,0,11.5,27.030709,78.061290,535.094296,1.393616,165.052917,3961.27,10653.244142,255677.8594,...,0.000000,0.0000,11224.012242,269376.2938,0.167719,1.0,1.958333,3.651666,9,19
1,1,11.5,27.542751,78.075041,523.986590,1.384431,165.856667,3980.56,10680.188596,256324.5263,...,0.000000,0.0000,11251.370396,270032.8895,0.181787,1.0,1.958333,3.996667,9,19
2,2,11.5,26.678482,77.590048,539.236170,1.392745,170.570833,4093.70,10655.177954,255724.2709,...,0.000000,0.0000,11224.980754,269399.5381,0.265921,1.0,1.958333,2.736667,9,19
3,3,11.5,26.157335,77.366091,760.150330,1.401298,165.856667,3980.56,10680.188596,256324.5263,...,0.000000,0.0000,11251.370396,270032.8895,0.423650,1.0,1.958333,2.450000,9,19
4,4,11.5,26.522891,78.131650,539.535758,1.403796,163.981250,3935.55,10659.561262,255829.4703,...,0.000000,0.0000,11229.639863,269511.3567,0.475272,1.0,1.958333,2.711666,9,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,23,11.5,27.901224,58.783863,457.965007,1.156525,1162.327917,27895.87,3522.632850,84543.1884,...,251.619063,6038.8575,4309.924463,103438.1871,64.875499,28.0,1.958333,8.825000,10,20
24,24,11.5,25.261621,59.967900,451.424429,1.189204,137.195417,3292.69,6109.429371,146626.3049,...,437.640938,10503.3825,7477.895308,179469.4874,74.002614,28.0,1.958333,3.663334,10,17
25,25,11.5,24.919758,60.240364,459.397150,1.191183,124.820000,2995.68,6109.042608,146617.0226,...,437.314583,10495.5500,7476.354792,179432.5150,76.342275,28.0,1.958333,2.461667,10,19
26,26,11.5,25.478958,60.134861,428.962500,1.171493,109.178333,2620.28,6109.687212,146632.4931,...,436.400792,10473.6190,7473.879204,179373.1009,82.621245,28.0,1.958333,4.195000,10,17


In [48]:
train_1 = pd.read_csv('./data/preprocessing/train_quantile0.25.csv')
test_1 = pd.read_csv('./data/preprocessing/test_quantile0.25.csv')
train_3 = pd.read_csv('./data/preprocessing/train_quantile0.75.csv')
test_3 = pd.read_csv('./data/preprocessing/test_quantile0.75.csv')
train_std = pd.read_csv('./data/preprocessing/train_std.csv')
test_std = pd.read_csv('./data/preprocessing/test_std.csv')
# train_mid = pd.read_csv('./data/preprocessing/train_median.csv')
# test_mid = pd.read_csv('./data/preprocessing/test_median.csv')
# train_sem = pd.read_csv('./data/preprocessing/train_sem.csv')
# test_sem = pd.read_csv('./data/preprocessing/test_sem.csv')

In [49]:
train_3.columns = [col+' 3' for col in train_3.columns]
test_3.columns = [col+' 3' for col in test_3.columns]
train_1.columns = [col+' 1' for col in train_1.columns]
test_1.columns = [col+' 1' for col in test_1.columns]
train_std.columns = [col+' std' for col in train_std.columns]
test_std.columns = [col+' std' for col in test_std.columns]
# train_mid.columns = [col+' mid' for col in train_mid.columns]
# test_mid.columns = [col+' mid' for col in test_mid.columns]
# train_sem.columns = [col+' sem' for col in train_sem.columns]
# test_sem.columns = [col+' sem' for col in test_sem.columns]

In [50]:
train = train.reset_index()
test  = test.reset_index()
train_3 = train_3.reset_index()
test_3 = test_3.reset_index()
train_1 = train_1.reset_index()
test_1 = test_1.reset_index()
train_std = train_std.reset_index()
test_std = test_std.reset_index()
# train_mid = train_mid.reset_index()
# test_mid = test_mid.reset_index()
# train_sem = train_sem.reset_index()
# test_sem = test_sem.reset_index()


In [51]:
train = pd.concat([train, train_1, train_3, train_std], axis=1)
test = pd.concat([test, test_1, test_3, test_std], axis=1)

In [52]:
train = train.drop(['index'], axis=1)
test = test.drop(['index'], axis=1)

In [53]:
train = pd.concat([train, p_train], axis=1)
test = pd.concat([test, p_test], axis=1)

In [54]:
accumulate(train, test, '누적분무량')
accumulate(train, test, '누적백색광량')
accumulate(train, test, '누적청색광량')
accumulate(train, test, '누적적색광량')
accumulate(train, test, '누적총광량')

In [55]:
def model_train(train, test, model):
    '''
    train shape = (784, 18), test shape = (140, 18)
    model => sklearn api model
    '''

    X = train.drop(['predicted_weight_g', 'Case', 'obs_time'], axis=1)
    y = train['predicted_weight_g']

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=113, shuffle=True)
    
    x_test = test.drop(['predicted_weight_g', 'Case', 'obs_time'], axis=1)
    
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)

    rmse = mean_squared_error(y_val, y_val_pred)**0.5
    print(f"validation rmse: {rmse}")
    
    x_train = train.drop(['predicted_weight_g', 'Case', 'obs_time'], axis=1)
    y_train = train['predicted_weight_g']
    
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    x_test['predicted_weight_g'] = y_pred
    submit = x_test[['DAT', 'predicted_weight_g']]
    submit['DAT'] = submit['DAT']+1
    all_target_list = sorted(glob.glob('./data/test_target/*.csv'))
    for idx, test_path in enumerate(all_target_list):
        submit_df = pd.read_csv(test_path)
        submit_df['predicted_weight_g'] = submit['predicted_weight_g'][idx*28:idx*28+28].values
        submit_df.to_csv(test_path, index=False)

In [56]:
xgb = XGBRegressor()
cat = CatBoostRegressor(verbose=100, early_stopping_rounds=50, task_type="GPU", random_seed=113)
lgbm = LGBMRegressor()
linear = LinearRegression()

model_train(train, test, cat)

Learning rate set to 0.037333
0:	learn: 40.7184677	total: 9.4ms	remaining: 9.39s
100:	learn: 7.8527340	total: 986ms	remaining: 8.78s
200:	learn: 5.6067142	total: 2s	remaining: 7.96s
300:	learn: 4.9664862	total: 2.99s	remaining: 6.94s
400:	learn: 4.6977367	total: 3.96s	remaining: 5.92s
500:	learn: 4.3434528	total: 4.87s	remaining: 4.85s
600:	learn: 4.1191460	total: 5.86s	remaining: 3.89s
700:	learn: 3.8981229	total: 6.85s	remaining: 2.92s
800:	learn: 3.5081550	total: 7.79s	remaining: 1.94s
900:	learn: 3.2652689	total: 8.72s	remaining: 958ms
999:	learn: 3.1221828	total: 9.69s	remaining: 0us
validation rmse: 9.12521808631475
Learning rate set to 0.039126
0:	learn: 40.3252323	total: 11.1ms	remaining: 11.1s
100:	learn: 6.9754696	total: 1.01s	remaining: 9.01s
200:	learn: 5.0427961	total: 1.99s	remaining: 7.9s
300:	learn: 4.5843875	total: 2.98s	remaining: 6.91s
400:	learn: 4.2436538	total: 3.98s	remaining: 5.95s
500:	learn: 4.0011902	total: 4.99s	remaining: 4.97s
600:	learn: 3.7480191	total: 

In [57]:
df_imp = pd.DataFrame({'imp':cat.feature_importances_}, index = cat.feature_names_)
df_imp = df_imp[df_imp.imp > 0].sort_values('imp').copy()
df_imp

,imp
"('ec관측치', 2.0)",0.000040
"('시간당백색광량', 1.0)",0.000047
"('시간당적색광량', 1.0)",0.000179
"('시간당백색광량', 4.0)",0.000292
"('시간당적색광량', 3.0)",0.000315
...,...
시간당총광량 std,5.102149
월간누적적색광량,10.297923
월간누적백색광량,10.431380
월간누적분무량,20.194500
